In [ ]:
import pandas as pd

df = pd.read_csv("data/ipa-dict-master/data/en_US.txt", sep="\t", header=None, keep_default_na=False)
df.columns = ["word_original", "ipa_original"]
display(df)

In [ ]:
import re

df["word"] = df["word_original"].apply(str.lower)

def normalize_ipa(ipa_original):
    result = re.search(r"/[^/]+/$", ipa_original)
    if result:
        return result.group()[1:-1]
    return ipa_original

df["ipa"] = df["ipa_original"].apply(normalize_ipa)

display(df)

In [ ]:
from enum import Enum
from dataclasses import dataclass

class Direction(Enum):
    TO_IPA = 1
    FROM_IPA = 2
    
class Sublanguage(Enum):
    WORD = 1
    IPA = 2

@dataclass
class Alphabet:
    locale: str
    sublanguage: Sublanguage
    
    def __repr__(self):
        return f"{self.locale}:{self.sublanguage.name}"

LOCALE = "en_US"
DIRECTION = Direction.TO_IPA

SRC_SUBLANGUAGE = Sublanguage.WORD if DIRECTION == Direction.TO_IPA else Sublanguage.IPA
TGT_SUBLANGUAGE = Sublanguage.IPA if DIRECTION == Direction.TO_IPA else Sublanguage.WORD

SRC_LANGUAGE = Alphabet(LOCALE, SRC_SUBLANGUAGE)
TGT_LANGUAGE = Alphabet(LOCALE, TGT_SUBLANGUAGE)

print(f"The model will translate {SRC_LANGUAGE} to {TGT_LANGUAGE}")

In [ ]:
def create_dataframe(filename):
    df = pd.read_csv(filename, sep="\t", header=None, keep_default_na=False)
    df.columns = ["word_original", "ipa_original"]
    df[Sublanguage.WORD.name] = df["word_original"].apply(str.lower)
    df[Sublanguage.IPA.name] = df["ipa_original"].apply(normalize_ipa)
    return df

df_en = create_dataframe("data/ipa-dict-master/data/en_US.txt")
display(df_en)

In [ ]:
from torch.utils.data import Dataset

class IpaDataset(Dataset):
    def __init__(self, df: pd.DataFrame, locale: str, direction: Direction):
        self._df = df
        self._direction = direction
        self._locale = locale
        # self._label = label
    
    def direction(self) -> Direction:
        return self._direction
    
    def locale(self) -> str:
        return self._locale
    
    def __len__(self):
        return len(self._df)
    
    def __getitem__(self, idx) -> list[str]:
        row = self._df.iloc[idx]
        if self._direction == Direction.TO_IPA:
            return [row[Sublanguage.WORD.name], row[Sublanguage.IPA.name]]
        else:
            return [row[Sublanguage.IPA.name], row[Sublanguage.WORD.name]]
    
    def __repr__(self):
        # opt_label = f"[{self._label}]" if self._label else ""
        return f"IpaDataset(size={len(self)}, locale={self.locale()}, dir={self.direction().name})"

ipa_en = IpaDataset(df_en, "en_US", Direction.TO_IPA)
print(ipa_en)

In [ ]:
import sys
print(sys.version_info)